In [ ]:
#mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#change the directory 
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/predicting_height_of_children')

In [ ]:
!ls 'depthmap'

####run this command to install all the dependencies

!pip install -r requirements.txt


In [ ]:
#import dependencies
import pandas as pd
import numpy as np
from PIL import Image
import ast
from collections import ChainMap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error , mean_squared_error

####run this command to rxport the dependencies


!pip freeze >> requirements.txt

####Initial setup if required

!mkdir '/content/drive/MyDrive/Colab Notebooks/predicting_height_of_children'
---
!unzip 'DS_Assignment 2.zip'
---
!rm -rf '__MACOSX'
---




# Explore the image files

In [ ]:
image= Image.open('depthmap/1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg')

In [ ]:
image.format, image.size, image.mode

In [ ]:
image.show()

In [ ]:
data =np.asarray(image)
data.shape

# Explore the height_pose dataframe

In [ ]:
height_pose_df = pd.read_excel('height_and_pose.xlsx')

In [ ]:
height_pose_df.head()

In [ ]:
type(height_pose_df.Pose[0])

In [ ]:
#to parse the string as json
parsed_json = ast.literal_eval(height_pose_df.Pose[0])
#parsed_json

In [ ]:
#json to python dictionary
data = dict(ChainMap(*parsed_json))
#data

In [ ]:
for key, value in data.items():
  print(key, ':', value)

In [ ]:
#@title Default title text
def append_value(dict_obj, key, value):
  '''
  This function takes dictionary object and a key-value pair,
  if the key exist in the current dict, then it appends the value as list,
  or else it creates a new key-value pair in the dict.
  '''
    # Check if key exist in dict or not
  if key in dict_obj:
    # Check if type of value of key is list or not
      if not isinstance(dict_obj[key], list):
          dict_obj[key] = [dict_obj[key]]
        # Append the value in list
      
      dict_obj[key].append(value)
  else:
    # As key is not in dict,
    # so, add key-value pair
      dict_obj[key] = value
    #print(dict_obj)
  return dict_obj

In [ ]:
def nested_dict_to_dataframe(pose_series):
    '''
    This function takes a pandas.Series of nested dicts as input,
    and returns pandas.Dataframe of single index as output
    '''
    coord_dict = {}
    for js in pose_series:
      parsed_json_data = ast.literal_eval(js)
      #print(parsed_json_data)
      dict_data = dict(ChainMap(*parsed_json_data))
      #print(dict_data)
      s = []
      for index, tup_obj in enumerate(dict_data['bbox_coordinates']):
        for t in tup_obj:
          s.append(t)
      #print(s)
      for index, value in enumerate(s):
          coord_dict = append_value(coord_dict, 'bbox_coordinates_{}'.format(index), value )
      #print(coord_dict)
      for value in dict_data['bbox_confidence_score']:
          coord_dict = append_value(coord_dict, 'bbox_confidence_score', value )
      #print('bcs')
      for x, y in enumerate(dict_data['key_points_coordinate']):
          for i,j in y.items(): 
              for m,n in j.items():  
                  coord_dict = append_value(coord_dict, 'key_points_coordinate_{}_{}'.format(i,m), n )
      #print('kpc')
      for x,y in enumerate(dict_data['key_points_prob']):
          for i, j in y.items():
              for m,n in j.items():
                  coord_dict = append_value(coord_dict, 'key_points_prob_score_{}'.format(i), n )
      coord_dict = append_value(coord_dict, 'body_pose_score', dict_data['body_pose_score'])
    #for i,j in coord_dict.items():
     # print(i, ':', len(j))
    df = pd.DataFrame(coord_dict)
    return df


In [ ]:
pose_df = nested_dict_to_dataframe(height_pose_df.Pose)

In [ ]:
pose_df

In [ ]:
#storing image data as numpy arrays
X_image_train = []
for img in height_pose_df['Depthmap Image']:
  image_path = 'depthmap/'+img+ '.jpg'
  image = Image.open(image_path)
  image_arr = np.asarray(image)
  X_image_train.append(image_arr)


In [ ]:
#flattening the multi dimensional arrays by replacing the array values with mean
X_image_train_flatten = []
for x in X_image_train:
  X_image_train_flatten.append(np.mean(x.flatten()))
X_image_train_flatten

In [ ]:
pose_df['Depthmap Image'] = X_image_train_flatten 

In [ ]:
height_df = height_pose_df['Height(cm)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pose_df, height_df, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
y_pred

In [ ]:
print('MAE: ', mean_absolute_error(y_test, y_pred))
print('MSE: ', mean_squared_error(y_test, y_pred))